<a href="https://colab.research.google.com/github/LukasStankevicius/Generating-abstractive-summaries-of-Lithuanian-news-articles-using-a-transformer-model/blob/main/Supplementary_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a supplementary code material for our work ["Generating abstractive summaries of Lithuanian news articles using a transformer model"](https://arxiv.org/abs/2105.03279)  
If you use this code for your research, please cite
```bibtex
@misc{stankevičius2021generating,
      title={Generating abstractive summaries of Lithuanian news articles using a transformer model}, 
      author={Lukas Stankevičius and Mantas Lukoševičius},
      year={2021},
      eprint={2105.03279},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}
```
# Contents:
* [Simple usage](#simple_usage)
* [Advanced usage](#advances_usage)
* [Automatic evaluation](#evaluation)
* [How we trained the tokenizer](#tokenizer)
* [How we trained the model](#training_model)
 * [Optimizer and scheduler](#opt)
 * [Data](#data)
 * [Final training script](#final)




Install libraries that we will need in this notebook:

In [ ]:
! pip install transformers==4.3 sentencepiece protobuf rouge-score PyStemmer

Import

In [ ]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Adafactor, Trainer
from rouge_score import rouge_scorer
import Stemmer
import sentencepiece as spm
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch


#Simple usage<a name='simple_usage'></a>

In [ ]:
name= "LukasStankevicius/t5-base-lithuanian-news-summaries-175"
my_pipeline = pipeline(task="text2text-generation", model=name, framework="pt")

Given the following article body from [15min](https://www.15min.lt/24sek/naujiena/lietuva/tarp-penkiu-rezultatyviausiu-tsrs-rinktines-visu-laiku-zaideju-trys-lietuviai-875-1380030):

In [ ]:
text = """
Latvijos krepšinio legenda Valdis Valteris pirmadienį socialiniame tinkle pasidalino statistika, kurios viršūnėje yra Arvydas Sabonis.
1982 metais TSRS rinktinėje debiutavęs 222 cm ūgio vidurio puolėjas su raudona apranga sužaidė 52 rungtynes, per kurias rinko po 15,6 taško. Tai pats aukščiausias rezultatyvumo vidurkis tarp visų sovietų komandai atstovavusių žaidėjų, skaičiuojant tuos, kurie sužaidė ne mažiau nei 50 rungtynių. Antras šioje rikiuotėje kitas buvęs Kauno „Žalgirio“ krepšininkas Rimas Kurtinaitis. Jis debiutavo TSRS rinktinėje vėliau nei Sabas, – 1984 metais, bet irgi sužaidė 52 mačus. R.Kurtinaitis pelnė po 15 taškų. 25-ių rezultatyviausių žaidėjų sąrašu pasidalinęs latvis V.Valteris, pelnęs po 13,8 taško, yra trečias.
Ketvirtas yra iš Kazachstano kilęs Valerijus Tichonenka, pelnęs po 13,7 taško per 79 rungtynes. Rezultatyviausią visų laikų TSRS rinktinės penketą uždaro Modestas Paulauskas. Lietuvos krepšinio legenda pelnė po 13,6 taško per 84 mačus.
Dešimtuke taip pat yra Oleksandras Volkovas (po 13,5 taško), Sergejus Belovas (12,7), Anatolijus Myškinas (po 12,3), Vladimiras Tkačenka (11,7) ir Aleksandras Salnikovas (11,4). Dvyliktas šiame sąraše yra Valdemaras Chomičius, vidutiniškai rinkęs po 10 taškų, o keturioliktas dar vienas buvęs žalgirietis Sergejus Jovaiša (po 9,8 taško). Šarūno Marčiulionio rezultatyvumo vidurkis turėjo būti aukštesnis, bet jis sužaidė mažiau nei 50 rungtynių. Kaip žinia, Lietuvai išsilaisvinus ir atkūrus Nepriklausomybę, visi minėti mūsų šalies krepšininkai, išskyrus karjerą jau baigusį M.Paulauską, užsivilko žalią aprangą ir atstovavo savo tėvynei.
A.Sabonis pagal rezultatyvumo vidurkį yra pirmas – jis Lietuvos rinktinei pelnė po 20 taškų. Antras pagal taškų vidurkį yra Artūras Karnišovas, rinkęs po 18,2 taško ir pelnęs iš viso daugiausiai taškų atstovaujant Lietuvos rinktinei (1453).
Tarp žaidėjų, kurie sužaidė bent po 50 oficialių rungtynių Lietuvos rinktinėje, trečią vietą užima Ramūnas Šiškauskas (po 12,9), ketvirtąją Linas Kleiza (po 12,7 taško), o penktas – Saulius Štombergas (po 11,1 taško). Daugiausiai rungtynių Lietuvos rinktinėje sužaidęs ir daugiausiai olimpinių medalių (3) su ja laimėjęs Gintaras Einikis rinko po 9,6 taško, o pirmajame trejete pagal rungtynių skaičių ir pelnytus taškus esantis Šarūnas Jasikevičius pelnė po 9,9 taško.
"""
text = ' '.join(text.strip().split())

The summary can be obtained by:

In [ ]:
my_pipeline(text)[0]["generated_text"]

'Lietuvos krepšinio federacijos (LKF) prezidento Arvydo Sabonio rezultatyvumo vidurkis yra aukščiausias tarp visų Sovietų Sąjungos rinktinėje atstovavusių žaidėjų, skaičiuojant tuos, kurie sužaidė bent po 50 oficialių rungtynių.'

#Advanced usage<a name='advances_usage'></a>

In [ ]:
name= "LukasStankevicius/t5-base-lithuanian-news-summaries-175"
tokenizer = T5Tokenizer.from_pretrained(name)
model = T5ForConditionalGeneration.from_pretrained(name)
def decode(x):
    return tokenizer.decode(x, skip_special_tokens=True)

Given the following article body from [15min](https://www.15min.lt/24sek/naujiena/lietuva/tarp-penkiu-rezultatyviausiu-tsrs-rinktines-visu-laiku-zaideju-trys-lietuviai-875-1380030):

In [ ]:
text = """
Latvijos krepšinio legenda Valdis Valteris pirmadienį socialiniame tinkle pasidalino statistika, kurios viršūnėje yra Arvydas Sabonis.
1982 metais TSRS rinktinėje debiutavęs 222 cm ūgio vidurio puolėjas su raudona apranga sužaidė 52 rungtynes, per kurias rinko po 15,6 taško. Tai pats aukščiausias rezultatyvumo vidurkis tarp visų sovietų komandai atstovavusių žaidėjų, skaičiuojant tuos, kurie sužaidė ne mažiau nei 50 rungtynių. Antras šioje rikiuotėje kitas buvęs Kauno „Žalgirio“ krepšininkas Rimas Kurtinaitis. Jis debiutavo TSRS rinktinėje vėliau nei Sabas, – 1984 metais, bet irgi sužaidė 52 mačus. R.Kurtinaitis pelnė po 15 taškų. 25-ių rezultatyviausių žaidėjų sąrašu pasidalinęs latvis V.Valteris, pelnęs po 13,8 taško, yra trečias.
Ketvirtas yra iš Kazachstano kilęs Valerijus Tichonenka, pelnęs po 13,7 taško per 79 rungtynes. Rezultatyviausią visų laikų TSRS rinktinės penketą uždaro Modestas Paulauskas. Lietuvos krepšinio legenda pelnė po 13,6 taško per 84 mačus.
Dešimtuke taip pat yra Oleksandras Volkovas (po 13,5 taško), Sergejus Belovas (12,7), Anatolijus Myškinas (po 12,3), Vladimiras Tkačenka (11,7) ir Aleksandras Salnikovas (11,4). Dvyliktas šiame sąraše yra Valdemaras Chomičius, vidutiniškai rinkęs po 10 taškų, o keturioliktas dar vienas buvęs žalgirietis Sergejus Jovaiša (po 9,8 taško). Šarūno Marčiulionio rezultatyvumo vidurkis turėjo būti aukštesnis, bet jis sužaidė mažiau nei 50 rungtynių. Kaip žinia, Lietuvai išsilaisvinus ir atkūrus Nepriklausomybę, visi minėti mūsų šalies krepšininkai, išskyrus karjerą jau baigusį M.Paulauską, užsivilko žalią aprangą ir atstovavo savo tėvynei.
A.Sabonis pagal rezultatyvumo vidurkį yra pirmas – jis Lietuvos rinktinei pelnė po 20 taškų. Antras pagal taškų vidurkį yra Artūras Karnišovas, rinkęs po 18,2 taško ir pelnęs iš viso daugiausiai taškų atstovaujant Lietuvos rinktinei (1453).
Tarp žaidėjų, kurie sužaidė bent po 50 oficialių rungtynių Lietuvos rinktinėje, trečią vietą užima Ramūnas Šiškauskas (po 12,9), ketvirtąją Linas Kleiza (po 12,7 taško), o penktas – Saulius Štombergas (po 11,1 taško). Daugiausiai rungtynių Lietuvos rinktinėje sužaidęs ir daugiausiai olimpinių medalių (3) su ja laimėjęs Gintaras Einikis rinko po 9,6 taško, o pirmajame trejete pagal rungtynių skaičių ir pelnytus taškus esantis Šarūnas Jasikevičius pelnė po 9,9 taško.
"""
text = ' '.join(text.strip().split())
input_dict = tokenizer(text,  padding=True, return_tensors="pt", return_attention_mask=True)

And generation parameters ([documentation](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate), [explanation](https://github.com/huggingface/blog/blob/master/notebooks/02_how_to_generate.ipynb)):

In [ ]:
g_kwargs = dict(max_length=512, num_beams=10, no_repeat_ngram_size=2, early_stopping=True)

The summary can be obtained by:

In [ ]:
output = model.generate(**input_dict, **g_kwargs)
list(map(decode, output.tolist()))[0]

'Lietuvos krepšinio federacijos (LKF) prezidento Arvydo Sabonio rezultatyvumo vidurkis yra aukščiausias tarp visų Sovietų Sąjungos rinktinėje atstovavusių žaidėjų, skaičiuojant tuos, kurie sužaidė bent po 50 oficialių rungtynių.'

If you do a lot of compute you can take advantage of GPU (of course if you have one). Obtain summary with:

In [ ]:
input_dict = {key:value.to("cuda:0") for key, value in input_dict.items()}
model = model.to("cuda:0")
output = model.generate(**input_dict, **g_kwargs)
list(map(decode, output.cpu().tolist()))[0]

'Lietuvos krepšinio federacijos (LKF) prezidento Arvydo Sabonio rezultatyvumo vidurkis yra aukščiausias tarp visų Sovietų Sąjungos rinktinėje atstovavusių žaidėjų, skaičiuojant tuos, kurie sužaidė bent po 50 oficialių rungtynių.'

# Automatic evaluation<a name='evaluation'></a>
We evaluated summaries with [ROUGE](https://www.aclweb.org/anthology/W04-1013/). It measures *n-gram* overlap between reference and generated texts. However, one should not completely trust it as the same meaning can be expressed by different words (*n-grams*). Yet it is almost the best we can do (automated and fast). Lithuanian language is quite rich with different word stem endings so we also "helped" ROUGE by stemming words.


Combining the two:

In [ ]:
class MyStemmer:
    def __init__(self):
        self.stemmer = Stemmer.Stemmer('lithuanian')

    def stem(self, token):
        return self.stemmer.stemWord(token)


class MyRougeScorer(rouge_scorer.RougeScorer):
    # I rewrite init to have different stemmer
    def __init__(self, rouge_types, use_stemmer=False):
        self.rouge_types = rouge_types
        self._stemmer = MyStemmer() if use_stemmer else None

Now, given the gold reference and generated summary:

In [ ]:
ground_truth = "Kai Lietuva dar buvo okupuota ir mūsų šalies krepšininkai privalėjo žaisti TSRS rinktinėje, keli jų buvo ryškūs lyderiai."
generated_text = "Lietuvos krepšinio federacijos (LKF) prezidento Arvydo Sabonio rezultatyvumo vidurkis yra aukščiausias tarp visų Sovietų Sąjungos rinktinėje atstovavusių žaidėjų, skaičiuojant tuos, kurie sužaidė bent po 50 oficialių rungtynių."

Let's calculate ROUGE:

In [ ]:
rouge_types = ['rouge1', 'rouge2', 'rougeL']
scorer = MyRougeScorer(rouge_types, use_stemmer=True)
score = scorer.score(ground_truth, generated_text)
print({s:score[s].fmeasure for s in rouge_types})

{'rouge1': 0.20689655172413793, 'rouge2': 0.03571428571428572, 'rougeL': 0.1724137931034483}


We monitored training by calculating ROUGE for 4096 validation pairs and noticed that after 250000 training steps our model started to overfit.


# How we trained the tokenizer<a name='tokenizer'></a>


Now we need a very big text file. Suppose we have one with over 1000000 lines in it and name it `"my_big_text_file.txt"`. Be warned that the following code requires a lot of memory (you can reduce number of lines sampled by lowering `input_sentence_size`) and can take several hours.

In [ ]:
default_kwargs = {
    "model_type": 'unigram', "pad_id": 0, "eos_id": 1, "unk_id": 2, "bos_id": -1, "pad_piece": '<pad>',
    "eos_piece": '</s>',
    "unk_piece": '<unk>', "input_sentence_size": 1000000, "max_sentencepiece_length": 64, "add_dummy_prefix": True
}
# more options are here: https://github.com/google/sentencepiece/blob/master/doc/options.md
spm.SentencePieceTrainer.train(
    input="my_big_text_file.txt",
    model_prefix="my_new_tokenizer",
    vocab_size=32000,
    split_by_whitespace=True,
    **default_kwargs
)
# normalization_rule_name=nmt_nfkc_cf if you want to lowercase

Now that our sentencepiece model is trained, let's put it in our `T5Tokenizer` from `transformers` library:

In [ ]:
tokenizer = T5Tokenizer("my_new_tokenizer.model", do_lower_case=False)
tokenizer._add_tokens(new_tokens=[f"<extra_id_{i}>" for i in range(100)] + ['</s>', '<pad>', '<unk>'],
                      special_tokens=True)
tokenizer.save_pretrained("MyNewT5Tokenizer")

So now you can load your trained tokenizer with:

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("MyNewT5Tokenizer")

# How we trained the model<a name='training_model'></a>

## Optimizer and scheduler<a name='opt'></a>
We used [T5](https://arxiv.org/abs/1910.10683) transformer model. It was originally trained using [Adafactor](https://arxiv.org/abs/1804.04235) optimizer. We used it with with 10 000 warm-up steps followed by inverse square root internal learning rate schedule. All of this is set internally, so we create `Dummy`, the fake learning rate scheduler.

In [ ]:
class Dummy:
    def step(self):
        return 1

    def get_last_lr(self):
        return [1]

    def state_dict(self):
        return {"dummy_key": 1}

    def load_state_dict(self, state_dict):
        pass

    def get_lr(self):
        return [1]

## Data<a name='data'></a>
Our training corpus consisted of over 6GB text file and was to big to load into the Colab RAM. So we:  
1. encoded it with our trained tokenizer - each string was converted to list of numbers from 0 to 32000;  
2. as our maximum number is 32000, we changed type of our lists to numpy arrays of type `uint16` which can contain integers from 0 to 65535;  
These "tricks" enabled us to load our pandas dataframe into Colab memory without memory errors.

For example purposes we will construct an example dataset

In [ ]:
# this will produce 10 rows with exactly the same line
df = pd.DataFrame.from_records(data=[("Čia yra naujienų straipsnio pagrindinė dalis.","O čia yra santrauka.")]*10, columns=["main", "summary"])
# load tokenizer
tokenizer = T5Tokenizer.from_pretrained("LukasStankevicius/t5-base-lithuanian-news-summaries-175")
# encode and reduce memory footprint with uint16 dtype
for col in ["main", "summary"]:
  df[col] = df[col].apply(tokenizer.encode, max_length=512, truncation=True)
  df[col] = df[col].apply(np.asarray, dtype=np.uint16)
# this will produce 400 000 rows with exactly the same line
df = pd.concat([df for i in range(40000)])
# shuffle rows
df = df.sample(frac=1)
# split to train and valid parts
df.iloc[:-4096].to_pickle("my_pandas_train_dataframe_pickle.gz")
df.iloc[-4096:].to_pickle("my_pandas_valid_dataframe_pickle.gz")

df.head()

,main,summary
8,"[902, 22, 835, 1881, 3502, 401, 4, 1]","[133, 211, 22, 1992, 892, 26, 4, 1]"
1,"[902, 22, 835, 1881, 3502, 401, 4, 1]","[133, 211, 22, 1992, 892, 26, 4, 1]"
5,"[902, 22, 835, 1881, 3502, 401, 4, 1]","[133, 211, 22, 1992, 892, 26, 4, 1]"
6,"[902, 22, 835, 1881, 3502, 401, 4, 1]","[133, 211, 22, 1992, 892, 26, 4, 1]"
5,"[902, 22, 835, 1881, 3502, 401, 4, 1]","[133, 211, 22, 1992, 892, 26, 4, 1]"


The following are dataset (loading pairs) and colloator (combining individual pairs into batches) classes:

In [ ]:
class My_Dataset(Dataset):
    def __init__(self, pickle_path):
        df = pd.read_pickle(pickle_path)
        self.examples = list(zip(df["main"], df["summary"]))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

class MyCollator:
    """
This collator is used for already encoded strings. It only truncates and pads
    """

    def __init__(self, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, list_of_tuples):
        train_x, train_y = zip(*list_of_tuples)
        # truncate
        train_x, train_y = [seq[: self.max_length] for seq in train_x], [seq[: self.max_length] for seq in train_y]

        # first the targets
        n_items = len(train_y)
        tt = self.tokenizer.pad({"input_ids": train_y}, padding=True,
                                return_tensors="pt", return_attention_mask=True)

        decoder_input_ids = torch.cat((torch.zeros(size=(n_items, 1), dtype=torch.int64), tt['input_ids']), axis=1)
        decoder_attention_mask = torch.cat((torch.ones(size=(n_items, 1), dtype=torch.int64), tt['attention_mask']),
                                           axis=1)

        decoder_input_ids = decoder_input_ids[:, :-1]  # one item is added at beginning, so one at the end to remove
        decoder_attention_mask = decoder_attention_mask[:, :-1]

        # now inputs
        inputs_dict = self.tokenizer.pad({"input_ids": train_x},  padding=True, return_tensors="pt",
                                         return_attention_mask=True)
        # finally combine the two
        return {"decoder_input_ids": decoder_input_ids, "decoder_attention_mask": decoder_attention_mask,
                "labels": tt['input_ids'], **inputs_dict}

## Final training script<a name='final'></a>
You will definitely need GPU here



In [ ]:
output_dir = "output_directory_for_my_model"

kwargs = TrainingArguments(
    fp16=True, per_device_train_batch_size=4, gradient_accumulation_steps=32,
    num_train_epochs=30, output_dir=output_dir, evaluation_strategy="steps", 
    per_device_eval_batch_size=4, max_grad_norm=None, logging_steps=2000, 
    save_steps=5000, eval_steps=2000, dataloader_num_workers=1, adafactor=True
)

tokenizer = T5Tokenizer.from_pretrained("LukasStankevicius/t5-base-lithuanian-news-summaries-175")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

trainer = Trainer(
    train_dataset=My_Dataset("my_pandas_train_dataframe_pickle.gz"), 
    eval_dataset=My_Dataset("my_pandas_valid_dataframe_pickle.gz"),
    model=model, data_collator=MyCollator(tokenizer), tokenizer=tokenizer,
    args=kwargs, 
    optimizers=(Adafactor((param for param in model.parameters() if param.requires_grad),
                           relative_step=True, warmup_init=True), Dummy()))

trainer.train()
trainer.save_model(output_dir)
trainer.state.save_to_json(output_dir + "/trainer_state.json")